# Цель и задачи модели
Давать абсолютно точное предсказание евро2024 на основании передовых научных знаний из обсласти астрологии


# Шаги которые были предприняты 

1) Сбор данных из открытых источников 

2) Парсинг данных с помощью python

3) Объединение данных в единую таблицу (с использованием расстояния Левенштейна) 

4) Конструирование признаков ( о признаках поговорим попозже) 

5) Оценка команд и создание прогноза 

# Дальнейшие действия 

1) Построение прогноза на каждый конеретный матч (мы должны фильтровать игроков)

2) Добавление дополнительных признаков (Ретрогадный Меркурий, совместимость Имен, даем бонусные очки за смешные имена) 


# Ключевые слова
#python #pandas #levenstein #web_scrapping #predictive_analysis #science


In [2]:
import pandas as pd 
import os 


os.listdir()


['players_commands.csv',
 'Zodiac_Football.ipynb',
 'players.csv',
 '.ipynb_checkpoints']

In [4]:
players_stats = pd.read_csv('players.csv')

In [5]:
display(players_stats.head())

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,...,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,...,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0
3,77,NaN,Lúcio,Lúcio,2012,506,lucio,Brazil,Brasília,Brazil,...,NaN,NaN,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/lucio/profil/s...,IT1,Juventus Football Club,200000.0,24500000.0
4,80,Tom,Starke,Tom Starke,2017,27,tom-starke,East Germany (GDR),Freital,Germany,...,right,194.0,NaN,IFM,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/tom-starke/pro...,L1,FC Bayern München,100000.0,3000000.0


In [6]:
players_stats.columns
cols = ['player_id', 'first_name', 'last_name', 'name', 'player_code', 'date_of_birth', 'country_of_citizenship', 'sub_position', 'position', 'last_season']

In [7]:
players_stats[cols].query("last_season>=2023")

,player_id,first_name,last_name,name,player_code,date_of_birth,country_of_citizenship,sub_position,position,last_season
188,3333,James,Milner,James Milner,james-milner,1986-01-04,England,Central Midfield,Midfield,2023
348,4391,Boy,Waterman,Boy Waterman,boy-waterman,1984-01-24,Netherlands,Goalkeeper,Goalkeeper,2023
417,5336,Anastasios,Tsokanis,Anastasios Tsokanis,anastasios-tsokanis,1991-05-02,Greece,Defensive Midfield,Midfield,2023
644,7161,Jonas,Hofmann,Jonas Hofmann,jonas-hofmann,1992-07-14,Germany,Attacking Midfield,Midfield,2023
723,7825,Pepe,Reina,Pepe Reina,pepe-reina,1982-08-31,Spain,Goalkeeper,Goalkeeper,2023
...,...,...,...,...,...,...,...,...,...,...
30519,1214946,Sid Ahmed,Aissaoui,Sid Ahmed Aissaoui,sid-ahmed-aissaoui,2005-01-11,Algeria,Central Midfield,Midfield,2023
30520,1225269,Erdem,Çalık,Erdem Çalık,erdem-calik,2006-06-05,Türkiye,Central Midfield,Midfield,2023
30521,1229924,Adama,Sidibeh,Adama Sidibeh,adama-sidibeh,1998-06-25,The Gambia,Centre-Forward,Attack,2023
30522,1240467,Artem,Prysyazhnyuk,Artem Prysyazhnyuk,artem-prysyazhnyuk,2005-07-17,Ukraine,Central Midfield,Midfield,2023


In [9]:
import requests 
r = requests.get("https://www.fourfourtwo.com/features/every-euro-2024-squad-all-the-teams-previewed#section-austria-euro-2024-squad")

In [10]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(r.text)



In [88]:
def find_country_squad_ul(soup):
    res = ''
    # Regex to find the phrase with any country name followed by 'Euro 2024 squad'
    pattern = re.compile(r'\b(\w+)\sEuro\s2024\ssquad\b', re.IGNORECASE)

    # Find all <p> elements that might contain the squad announcements
    p_elements = soup.find_all('p')
    total = []

    for p in p_elements:
        res = []

        if pattern.search(p.text):
            country = pattern.search(p.text).group(1)
#             print(p.get_text())
            
            # Initialize an empty list to store the strings from each <li>
            li_strings = []



            # Find the next <ul> element following the paragraph
            next_ul = p.find_next('ul')
            next_ul = p.find_next('ul')
            if next_ul:
                data = [x.text.strip() for x in next_ul]
            res.append(([country] * len(data), data))
        total.append(res)
    return total 

                

# Call the function with the soup object
res = find_country_squad_ul(soup)


Albania
Austria
Belgium
Croatia
Republic
Denmark
England
France
Georgia
Germany
Hungary
Italy
Netherlands
Poland
Belgium
Romania
Scotland
Serbia
Slovakia
Slovenia
Spain
Switzerland
Turkey
Ukraine


In [109]:
# Unpacking the lists into variables
res_df = pd.DataFrame()
for i in range(len(res)):
    try:
        countries, player_details = res[i][0][0], res[i][0][1]

        # Create DataFrame
        df = pd.DataFrame({
            "Country": countries,
            "Player Details": player_details
        })
        res_df = pd.concat([res_df, df], axis=0)
    except IndexError:
        pass


dataframe = res_df.reset_index(drop=True)
# Regular expression to capture position, name, and club
pattern = r'^(?P<Position>[A-Z]{2}): (?P<Name>[\w\s]+) \((?P<Club>[\w\s-]+)\)$'

# Extract components into new columns
dataframe[['Position', 'Name', 'Club']] = dataframe['Player Details'].str.extract(pattern)
dataframe.drop(columns='Player Details', inplace=True)

In [118]:
dataframe.to_csv('players_commands.csv', index=False)

In [11]:
dataframe = pd.read_csv('players_commands.csv')


In [12]:
dataframe

,Country,Position,Name,Club
0,Albania,GK,Etrit Berisha,Empoli
1,Albania,GK,Thomas Strakosha,Brentford
2,Albania,GK,Elhan Kastrati,Cittadella
3,Albania,GK,Simon Simoni,Eintracht Frankfurt
4,Albania,DF,Berat Gjimshiti,Atalanta
...,...,...,...,...
682,Ukraine,MF,Volodymyr Brazhko,Dynamo Kyiv
683,Ukraine,FW,Roman Yaremchuk,Valencia
684,Ukraine,FW,Artem Dovbyk,Girona
685,Ukraine,FW,Vladyslav Vanat,Dynamo Kyiv


In [13]:
# Function to preprocess names (lowercase and sort words)
from fuzzywuzzy import process

def preprocess_name(name):
    return ' '.join(sorted(str(name).lower().split()))

# Apply preprocessing
dataframe['Processed Name'] = dataframe['Name'].apply(preprocess_name)
players_stats['Processed Name'] = players_stats['name'].apply(preprocess_name)

# Create a mapping from processed names in dataframe to processed names in players_stats
name_mapping = {}
for processed_name in dataframe['Processed Name']:
    # Find the best match in players_stats using a score cutoff for better accuracy
    match = process.extractOne(processed_name, players_stats['Processed Name'], score_cutoff=90)
    if match:
        name_mapping[processed_name] = match[0]

# Map the names in dataframe to their best matches in players_stats
dataframe['Matched Name'] = dataframe['Processed Name'].map(name_mapping)

# Merge based on the best matches found
result_df = pd.merge(dataframe, players_stats, left_on='Matched Name', right_on='Processed Name', how='left')
result_df.drop(columns=['Processed Name_x', 'Processed Name_y', 'Matched Name'], inplace=True)


In [14]:
data = result_df.drop_duplicates(subset='url', keep='first').dropna(subset='date_of_birth')

In [16]:
data.to_csv('result_data.csv', index=False)

In [17]:
data = pd.read_csv('result_data.csv')

In [139]:
def get_zodiac_sign(date_str):
    # Extract month and day from the date string
    month, day = int(date_str[5:7]), int(date_str[8:10])
    
    # Determine the zodiac sign based on month and day, returning the name in Russian
    if (month == 3 and day >= 21) or (month == 4 and day <= 19):
        return 'Овен'  # Aries
    elif (month == 4 and day >= 20) or (month == 5 and day <= 20):
        return 'Телец'  # Taurus
    elif (month == 5 and day >= 21) or (month == 6 and day <= 20):
        return 'Близнецы'  # Gemini
    elif (month == 6 and day >= 21) or (month == 7 and day <= 22):
        return 'Рак'  # Cancer
    elif (month == 7 and day >= 23) or (month == 8 and day <= 22):
        return 'Лев'  # Leo
    elif (month == 8 and day >= 23) or (month == 9 and day <= 22):
        return 'Дева'  # Virgo
    elif (month == 9 and day >= 23) or (month == 10 and day <= 22):
        return 'Весы'  # Libra
    elif (month == 10 and day >= 23) or (month == 11 and day <= 21):
        return 'Скорпион'  # Scorpio
    elif (month == 11 and day >= 22) or (month == 12 and day <= 21):
        return 'Стрелец'  # Sagittarius
    elif (month == 12 and day >= 22) or (month == 1 and day <= 19):
        return 'Козерог'  # Capricorn
    elif (month == 1 and day >= 20) or (month == 2 and day <= 18):
        return 'Водолей'  # Aquarius
    elif (month == 2 and day >= 19) or (month == 3 and day <= 20):
        return 'Рыбы'  # Pisces


In [140]:
data['zodiac_sign'] = data['date_of_birth'].apply(get_zodiac_sign)


In [142]:
zodiac_signs = ["Овен", "Лев", "Лев", "Дева", "Близнецы", "Водолей", "Овен", "Близнецы", "Лев", "Овен", "Весы", "Овен", "Лев", "Скорпион", "Козерог", "Стрелец"]

# Use a dictionary to count occurrences of each sign
from collections import Counter
zodiac_counts = Counter(zodiac_signs)

# Convert the counter object to a dictionary if necessary
zodiac_weights = dict(zodiac_counts)
print(zodiac_weights)


{'Овен': 4, 'Лев': 4, 'Дева': 1, 'Близнецы': 2, 'Водолей': 1, 'Весы': 1, 'Скорпион': 1, 'Козерог': 1, 'Стрелец': 1}


In [150]:
# Map the zodiac weights to the DataFrame
data['Weight_Zodiac'] = data['zodiac_sign'].map(zodiac_weights)
data['Weight_Zodiac'] = data['Weight_Zodiac'].fillna(0) + 1

data

,Country,Position,Name,Club,player_id,first_name,last_name,name,last_season,current_club_id,...,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur,zodiac_sign,Weight_Zodiac
0,Albania,GK,Etrit Berisha,Empoli,77825.0,Etrit,Berisha,Etrit Berisha,2023.0,749.0,...,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/etrit-berisha/...,IT1,Empoli Football Club S.r.l.,500000.0,7000000.0,Рыбы,1.0
1,Albania,GK,Thomas Strakosha,Brentford,222209.0,Thomas,Strakosha,Thomas Strakosha,2023.0,1148.0,...,2026-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/thomas-strakos...,GB1,Brentford Football Club,3000000.0,25000000.0,Рыбы,1.0
3,Albania,GK,Simon Simoni,Eintracht Frankfurt,726120.0,Simon,Simoni,Simon Simoni,2023.0,24.0,...,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/simon-simoni/p...,L1,Eintracht Frankfurt Fußball AG,200000.0,250000.0,Рак,1.0
4,Albania,DF,Berat Gjimshiti,Atalanta,165513.0,Berat,Djimsiti,Berat Djimsiti,2023.0,800.0,...,2025-06-30 00:00:00,LITCHI AG,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/berat-djimsiti...,IT1,Atalanta Bergamasca Calcio S.p.a.,10000000.0,22000000.0,Рыбы,1.0
5,Albania,DF,Elseid Hysaj,Lazio,200056.0,Elseid,Hysaj,Elseid Hysaj,2023.0,398.0,...,2025-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/elseid-hysaj/p...,IT1,Società Sportiva Lazio S.p.A.,3000000.0,25000000.0,Водолей,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,Ukraine,MF,Volodymyr Brazhko,Dynamo Kyiv,669306.0,Volodymyr,Brazhko,Volodymyr Brazhko,2023.0,338.0,...,2028-06-30 00:00:00,ProStar,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/volodymyr-braz...,UKR1,Futbolniy Klub Dynamo Kyiv,2500000.0,2500000.0,Водолей,2.0
737,Ukraine,FW,Roman Yaremchuk,Valencia,221958.0,Roman,Yaremchuk,Roman Yaremchuk,2023.0,1049.0,...,2024-06-30 00:00:00,Vision Foot,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-yaremchu...,ES1,Valencia Club de Fútbol S. A. D.,4000000.0,17500000.0,Стрелец,2.0
738,Ukraine,FW,Artem Dovbyk,Girona,343537.0,Artem,Dovbyk,Artem Dovbyk,2023.0,12321.0,...,2028-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/artem-dovbyk/p...,ES1,Girona Fútbol Club S. A. D.,30000000.0,30000000.0,Рак,1.0
739,Ukraine,FW,Vladyslav Vanat,Dynamo Kyiv,631821.0,Vladyslav,Vanat,Vladyslav Vanat,2023.0,338.0,...,2027-06-30 00:00:00,ProStar,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/vladyslav-vana...,UKR1,Futbolniy Klub Dynamo Kyiv,7000000.0,7000000.0,Козерог,2.0


In [151]:
# Словарь астрологических сочетаний
good_combinations = {
    ('Овен', 'Лев'), ('Лев', 'Овен'),
    ('Водолей', 'Близнецы'), ('Близнецы', 'Водолей'),
    ('Скорпион', 'Рак'), ('Рак', 'Скорпион'),
    ('Телец', 'Козерог'), ('Козерог', 'Телец'),
    ('Стрелец', 'Весы'), ('Весы', 'Стрелец')
}


In [153]:
data.head()

,Country,Position,Name,Club,player_id,first_name,last_name,name,last_season,current_club_id,...,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur,zodiac_sign,Weight_Zodiac
0,Albania,GK,Etrit Berisha,Empoli,77825.0,Etrit,Berisha,Etrit Berisha,2023.0,749.0,...,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/etrit-berisha/...,IT1,Empoli Football Club S.r.l.,500000.0,7000000.0,Рыбы,1.0
1,Albania,GK,Thomas Strakosha,Brentford,222209.0,Thomas,Strakosha,Thomas Strakosha,2023.0,1148.0,...,2026-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/thomas-strakos...,GB1,Brentford Football Club,3000000.0,25000000.0,Рыбы,1.0
3,Albania,GK,Simon Simoni,Eintracht Frankfurt,726120.0,Simon,Simoni,Simon Simoni,2023.0,24.0,...,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/simon-simoni/p...,L1,Eintracht Frankfurt Fußball AG,200000.0,250000.0,Рак,1.0
4,Albania,DF,Berat Gjimshiti,Atalanta,165513.0,Berat,Djimsiti,Berat Djimsiti,2023.0,800.0,...,2025-06-30 00:00:00,LITCHI AG,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/berat-djimsiti...,IT1,Atalanta Bergamasca Calcio S.p.a.,10000000.0,22000000.0,Рыбы,1.0
5,Albania,DF,Elseid Hysaj,Lazio,200056.0,Elseid,Hysaj,Elseid Hysaj,2023.0,398.0,...,2025-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/elseid-hysaj/p...,IT1,Società Sportiva Lazio S.p.A.,3000000.0,25000000.0,Водолей,2.0


In [182]:
# Функция для подсчета удачных сочетаний
def calculate_compatibility_scores(group):
    scores = 0
    # Перебираем все пары игроков в команде
    for i in range(len(group)):
        for j in range(i + 1, len(group)):
            if (group.iloc[i]['zodiac_sign'], group.iloc[j]['zodiac_sign']) in good_combinations:
                scores += 1
    return scores

# Применяем функцию к каждой сборной

compatibility_df = data.groupby('Country').apply(calculate_compatibility_scores).reset_index().sort_values(by=0, ascending=False)
compatibility_df.columns = ['Country', 'Score']
compatibility_df['Rank'] = compatibility_df['Score'].rank(ascending=True)
compatibility_df

,Country,Score,Rank
2,Belgium,86,23.0
1,Austria,41,22.0
21,Turkey,38,21.0
19,Spain,36,20.0
5,England,33,19.0
15,Scotland,32,18.0
10,Italy,31,17.0
20,Switzerland,30,16.0
11,Netherlands,24,15.0
3,Croatia,22,13.0


In [183]:
individual_df = data.groupby('Country').agg('sum')['Weight_Zodiac'].reset_index().sort_values(by='Weight_Zodiac', ascending=False)
individual_df['Rank'] = individual_df['Weight_Zodiac'].rank(ascending=True)
individual_df

/var/folders/4m/qtn55h5157d6r7mt5yc6frt80000gp/T/ipykernel_177/2619571946.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  individual_df = data.groupby('Country').agg('sum')['Weight_Zodiac'].reset_index().sort_values(by='Weight_Zodiac', ascending=False)


,Country,Weight_Zodiac,Rank
2,Belgium,107.0,23.0
1,Austria,90.0,22.0
21,Turkey,76.0,21.0
10,Italy,75.0,20.0
15,Scotland,65.0,18.5
12,Poland,65.0,18.5
5,England,62.0,17.0
11,Netherlands,61.0,15.5
20,Switzerland,61.0,15.5
6,France,60.0,14.0


In [177]:
height_in_cm_df = data.groupby('Country').agg('mean')['height_in_cm'].reset_index().sort_values(by='height_in_cm', ascending=False)
height_in_cm_df['Rank'] = height_in_cm_df['height_in_cm'].rank(ascending=True)
height_in_cm_df

/var/folders/4m/qtn55h5157d6r7mt5yc6frt80000gp/T/ipykernel_177/3323972215.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  height_in_cm_df = data.groupby('Country').agg('mean')['height_in_cm'].reset_index().sort_values(by='height_in_cm', ascending=False)


,Country,height_in_cm,Rank
16,Serbia,186.615385,23.0
4,Denmark,186.173913,22.0
8,Germany,186.160000,21.0
7,Georgia,185.695652,20.0
11,Netherlands,185.600000,19.0
6,France,185.304348,18.0
18,Slovenia,185.227273,17.0
13,Republic,185.150000,16.0
12,Poland,185.037037,15.0
0,Albania,184.478261,14.0


In [185]:
babki_df = data.groupby('Country').agg('sum')['market_value_in_eur'].reset_index().sort_values(by='market_value_in_eur', ascending=False)
babki_df['Rank'] = babki_df['market_value_in_eur'].rank(ascending=True)
babki_df

/var/folders/4m/qtn55h5157d6r7mt5yc6frt80000gp/T/ipykernel_177/3238215822.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  babki_df = data.groupby('Country').agg('sum')['market_value_in_eur'].reset_index().sort_values(by='market_value_in_eur', ascending=False)


,Country,market_value_in_eur,Rank
5,England,1.720000e+09,23.0
2,Belgium,1.613825e+09,22.0
6,France,1.177000e+09,21.0
19,Spain,8.934500e+08,20.0
11,Netherlands,8.110000e+08,19.0
10,Italy,7.586000e+08,18.0
8,Germany,7.030250e+08,17.0
4,Denmark,3.835000e+08,16.0
21,Turkey,3.676000e+08,15.0
22,Ukraine,3.107000e+08,14.0


In [189]:
total = compatibility_df.merge(individual_df, how='left', left_on='Country', right_on='Country')\
.merge(height_in_cm_df, how='left', left_on='Country', right_on='Country', suffixes=('_a', '_b'))\
.merge(babki_df, how='left', left_on='Country', right_on='Country', suffixes=('_m', '_n'))
total['Rank_Final'] = total['Rank_x'] + total['Rank_y'] + total['Rank_m'] + total['Rank_n']
total

,Country,Score,Rank_x,Weight_Zodiac,Rank_y,height_in_cm,Rank_m,market_value_in_eur,Rank_n,Rank_Final
0,Belgium,86,23.0,107.0,23.0,182.946429,2.0,1.613825e+09,22.0,70.0
1,Austria,41,22.0,90.0,22.0,183.225806,4.0,2.214000e+08,11.0,59.0
2,Turkey,38,21.0,76.0,21.0,184.029412,11.0,3.676000e+08,15.0,68.0
3,Spain,36,20.0,55.0,12.0,182.076923,1.0,8.934500e+08,20.0,53.0
4,England,33,19.0,62.0,17.0,183.593750,6.0,1.720000e+09,23.0,65.0
5,Scotland,32,18.0,65.0,18.5,183.000000,3.0,1.730000e+08,8.0,47.5
6,Italy,31,17.0,75.0,20.0,183.939394,10.0,7.586000e+08,18.0,65.0
7,Switzerland,30,16.0,61.0,15.5,183.785714,8.0,2.553500e+08,13.0,52.5
8,Netherlands,24,15.0,61.0,15.5,185.600000,19.0,8.110000e+08,19.0,68.5
9,Croatia,22,13.0,46.0,8.0,183.916667,9.0,2.498500e+08,12.0,42.0


In [190]:
-10790-48208-8707-13994+228973+416885

564159